# International Trade Data - Census API

The purpose of this project is to download data from the Monthly International Trade Dataset (MITD) by using Census API.

To make more than 500 API calls a day, Census API requires users to sign up for an API Key. To get one, register [here](http://api.census.gov/data/key_signup.html).

## About the Data

In this project, we'll be downloading 2-digit Harmonized System (HS) data. HS is an international classification system administered by the World Customs Organization.

In [1]:
import numpy as np
import pandas as pd
import requests

# no decimal places
pd.options.display.float_format = '{:,.0f}'.format
# show more rows
pd.set_option('display.max_rows', 500)

### Commodities

Commodities list consists of **97** unique 2-digit HS codes. An example would be 01, which stands for live animals.
The list used as a reference point is obtained from the [U.S. Census Bureau](https://www.census.gov/foreign-trade/schedules/b/2020/index.html). The existing commodity descriptions in this list will be replaced with simplified and easier-to-read descriptions as defined in the Houston Global publication issued by the Greater Houston Partnership.

In [2]:
commodities = pd.read_csv('C:/Users/Berina/Desktop/Trade Data - API/hs2_commodities.csv', header=None, dtype=object)
comm_alter_names = pd.read_csv('C:/Users/Berina/Desktop/Trade Data - API/commodities_alter_names.csv', 
                               header=None, dtype=object)

# initially comm_alter_names has 99 rows and 4 columns
# delete the two columns with NaN values
comm_alter_names.drop(labels=[2,3], axis=1,inplace=True)
# extract the two-digit codes from the first column
comm_alter_names['COMMODITY_CODE'] = comm_alter_names.iloc[:,0].str[0:2]
# drop rows with null values - this brings the list down to 97 rows
comm_alter_names = comm_alter_names[comm_alter_names['COMMODITY_CODE'].notnull()]
# delete the first column and rename the second one
comm_alter_names.drop(labels=0, axis=1, inplace=True)
comm_alter_names.rename(columns={1: 'COMMODITY'}, inplace=True)

# update column names for the commodities dataframe
commodities.columns = ['COMMODITY_CODE', 'LONG_NAME']

# combine the two files as well as verify the alternative list has all the right matches
commodities = pd.merge(left=commodities, right=comm_alter_names, on='COMMODITY_CODE', how='left')
# commodities.info() --> all three columns have 97 non-null object entries

# drop the long name
commodities.drop(labels='LONG_NAME', axis=1, inplace=True)

# prepare a list of commodity codes needed for API calls
commodity_codes = commodities.loc[:,'COMMODITY_CODE'].tolist()

### Districts

District codes are 2-digit unique identifiers for a total of **44** U.S. districts. The codes range from 01 to 55. An example district code is 53, which stands for Houston-Galveston, TX. The list of districts is also sourced from the [U.S.Census Bureau](https://www.census.gov/foreign-trade/schedules/d/dist2.txt).

In [3]:
districts = pd.read_csv('C:/Users/Berina/Desktop/Trade Data - API/districts.csv', header=None, dtype=object)
# rename the columns
districts.columns = ['DISTRICT_CODE', 'DISTRICT']

# prepare a list of district codes needed for API calls
district_codes = districts.loc[:,'DISTRICT_CODE'].tolist()

### Countries

Country codes are 4-digit unique identifiers and some examples include 1000, 1010, 1220, etc. For example, 1220 stands for Canada. This list consists of **241** countries and it is updated to reflect current geopolitical factors. The original list can be found [here](https://www.census.gov/foreign-trade/schedules/c/country2.txt).
Given that the countries list is longer than both commodities and districts lists, it will not be used for builing API queries. The reason it's necessary, however, is that we will be adding country names to the finalized dataset.

In [4]:
countries = pd.read_csv('C:/Users/Berina/Desktop/Trade Data - API/countries.csv', header=None, dtype=object)

# remove the last column, the country abbreviations, since it is not necessary
countries.drop(labels=2, axis=1, inplace=True)
# set column names
country_cols = ['COUNTRY_CODE', 'COUNTRY']
countries.columns = country_cols

# abbreviate long country names
#   Falkland Islands (Islas Malvinas)                          --> Falkland Islands
#   Denmark, except Greenland                                  --> Denmark
#   Germany (Federal Repulic of Germany)                       --> Germany
#   Moldova (Repulic of Moldova)                               --> Moldova
#   Holy See (Vatican City)                                    --> Vatican City
#   Syria (Syrian Arab Republic)                               --> Syria
#   Gaza Strip administered by Israel                          --> Gaza Strip
#   West Bank administered by Israel                           --> West Bank
#   Yemen (Republic of Yemen)                                  --> Yemen
#   Burma (Myanmar)                                            --> Myanmar
#   Laos (Lao People's Democratic Republic)                    --> Laos
#   North Korea (democratic People's Republic of Korea)        --> North Korea
#   South Korea (Republic of Korea)                            --> South Korea
#   Samoa (Western Samoa)                                      --> Samoa
#   Micronesia, Federated States of                            --> Micronesia
#   Congo, Republic of the Congo                               --> Congo-Brazzaville
#   Congo, Democratic Republic of the Congo (formerly Za[ire)] --> Congo-Kinshasa
#   Tanzania (United Republic of Tanzania)                     --> Tanzania
#   Christmas Island (in the Indian Ocean)                     --> Christmas Island

# make a dictionary with shorter country name alternatives
updated_country_names = {'Falkland Islands (Islas Malvinas)': 'Falkland Islands',
                         'Denmark, except Greenland': 'Denmark',
                         'Germany (Federal Republic of Germany)': 'Germany',
                         'Moldova (Republic of Moldova)': 'Moldova',
                         'Holy See (Vatican City)': 'Vatican City',
                         'Syria (Syrian Arab Republic)': 'Syria',
                         'Gaza Strip administered by Israel': 'Gaza Strip',
                         'West Bank administered by Israel': 'West Bank',
                         'Yemen (Republic of Yemen)': 'Yemen',
                         'Burma (Myanmar)': 'Myanmar',
                         'Laos (Lao People\'s Democratic Republic)': 'Laos',
                         'North Korea (Democratic People\'s Republic of Korea)': 'North Korea',
                         'South Korea (Republic of Korea)': 'South Korea',
                         'Samoa (Western Samoa)': 'Samoa',
                         'Micronesia, Federated States of': 'Micronesia',
                         'Congo, Republic of the Congo': 'Congo-Brazzaville',
                         'Congo, Democratic Republic of the Congo (formerly Za': 'Congo-Kinshasa',
                         'Tanzania (United Republic of Tanzania)': 'Tanzania',
                         'Christmas Island (in the Indian Ocean)': 'Christmas Island'}

# clean any unwanted whitespace
countries.loc[:,'COUNTRY_CODE'] = countries.loc[:,'COUNTRY_CODE'].str.strip()
countries.loc[:,'COUNTRY'] = countries.loc[:,'COUNTRY'].str.strip()
# replace the long names
countries.loc[:,'COUNTRY'].replace(to_replace=updated_country_names, inplace=True)

### Monthly International Trade Data API Structure

API for MITD data has two endpoints: exports and imports. There are several datasets available for each, but as mentioned previously, we will be using the HS data. This means that although both exports and imports have an HS dataset available, a separate API call will have to be made to access each. 

The general structure of the API call includes the UNIFORM RESOURCE IDENTIFIERS (URIs) of the MITD endpoints
    ``https://api.census.gov/data/timeseries/intltrade/exports/hs?get=``, where hs stands for the HS dataset. This is an example for the exports endpoint. Getting imports is as simple as typing imports instead of exports.
    
To complete the rest of the API call, we use parameters. There are more than 50 available parameters for both exports and imports. For this project, we will use the following:
            
            CTY_CODE        --> country code
            ALL_VAL_MO      --> total value, monthly
            AIR_VAL_MO      --> air value, monthly
            AIR_WGT_MO      --> air shipping weight, monthly
            VES_VAL_MO      --> vessel value, monthly
            VES_WGT_MO      --> vessel shipping weight, monthly
            SUMMARY_LVL=DET --> selecting individual countries, not predefined groups
            time            --> YYYY-MM format, or from+YYYY-MM+to+YYYY-MM if multiple months
            E(I)_COMMODITY  --> export/import commodity code
            DISTRICT        --> district code
            
For more information on how to build MITD API queries, refer to the [International Trade Data - API User Guide](https://www.census.gov/foreign-trade/reference/guides/Guide%20to%20International%20Trade%20Datasets.pdf) published by the U.S. Census Bureau.


### Getting the Data
***

In [5]:
# shorten the lists to test the code on a smaller sample size
# first five commodity codes
alter_comm_codes = commodity_codes[0:5]
# last five district codes
alter_dist_codes = district_codes[-5:]

In [6]:
key = 'c64e82bd341ac24cc8223afd0458afb0f3436c66' # insert your API key here

# it is advisable to split the time interval into smaller pieces as requesting too much data at once may cause performance 
# issues; in general, the API can handle a large number of smaller calls better than a few large data calls
time = ['from+2018-01+to+2018-12', 'from+2019-01+to+2019-12']

# Fixed URI parts 
exports_endpoint = "https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key="
imports_endpoint = "https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key="

# control variable used to identify the first successful API call
status_success = 0

for t in time:
    for commodity in alter_comm_codes:
        for district in alter_dist_codes:
            # build the custom API call
            exp_api = exports_endpoint + key + '&time=' + t + '&E_COMMODITY=' + commodity + '&DISTRICT=' + district
            imp_api = imports_endpoint + key + '&time=' + t + '&I_COMMODITY=' + commodity + '&DISTRICT=' + district
            # Requesting API
            for api in [exp_api, imp_api]:
                api_response = requests.get(api)
                print(api_response.status_code)
                print(api)
                # status code 200 means the server returned a result
                # 204 stands for a good request but no records for the given combination of parameters
                # everything else reads trouble
                if api_response.status_code == 200:
                    status_success += 1
                    # print('Commodity Code: ', commodity, '         District Code: ', district, '\n')
                    # save the first successful query result as a dataframe
                    if status_success == 1:
                        data = pd.DataFrame(api_response.json())
                        # denote the data type
                        data['type'] = 'Exports' if api == exp_api else 'Imports'
                    # concatenate subsequent successful query results
                    else:
                        data_2 = pd.DataFrame(api_response.json())
                        data_2['type'] = 'Exports' if api == exp_api else 'Imports'
                        data = pd.concat([data, data_2.iloc[1:,:]])                    
                    

204
https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2018-01+to+2018-12&E_COMMODITY=01&DISTRICT=51
204
https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2018-01+to+2018-12&I_COMMODITY=01&DISTRICT=51
200
https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2018-01+to+2018-12&E_COMMODITY=01&DISTRICT=52
Commodity Code:  01          District Code:  52
200
https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+20

200
https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2018-01+to+2018-12&I_COMMODITY=03&DISTRICT=55
Commodity Code:  03          District Code:  55
204
https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2018-01+to+2018-12&E_COMMODITY=04&DISTRICT=51
204
https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2018-01+to+2018-12&I_COMMODITY=04&DISTRICT=51
200
https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+20

200
https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2019-01+to+2019-12&E_COMMODITY=01&DISTRICT=55
Commodity Code:  01          District Code:  55
200
https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2019-01+to+2019-12&I_COMMODITY=01&DISTRICT=55
Commodity Code:  01          District Code:  55
204
https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2019-01+to+2019-12&E_COMMODITY=02&DISTRICT=51
204
https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e8

200
https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2019-01+to+2019-12&I_COMMODITY=04&DISTRICT=54
Commodity Code:  04          District Code:  54
200
https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2019-01+to+2019-12&E_COMMODITY=04&DISTRICT=55
Commodity Code:  04          District Code:  55
200
https://api.census.gov/data/timeseries/intltrade/imports/hs?get=CTY_CODE,GEN_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO&SUMMARY_LVL=DET&key=c64e82bd341ac24cc8223afd0458afb0f3436c66&time=from+2019-01+to+2019-12&I_COMMODITY=04&DISTRICT=55
Commodity Code:  04          District Code:  55
204
https://api.census.gov/data/timeseries/intltrade/exports/hs?get=CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO

### Cleaning the Data
***
To prepare the data for use, we need to do several steps including assigning meaningful column names, deleting unecessary columns and rows, adding year and month columns, adding country, commodity and district names and converting intrinsically numerical variables from strings to numerals.

In [7]:
data.head()

,0,1,2,3,4,5,6,7,8,9,type
0,CTY_CODE,ALL_VAL_MO,AIR_VAL_MO,AIR_WGT_MO,VES_VAL_MO,VES_WGT_MO,SUMMARY_LVL,time,E_COMMODITY,DISTRICT,Exports
1,-,6317585,6302719,156064,14866,2981,DET,2018-01,01,52,Exports
2,-,4527629,4392416,187891,135213,31478,DET,2018-02,01,52,Exports
3,-,8953527,8632870,118855,320657,137754,DET,2018-03,01,52,Exports
4,-,14447059,14384371,290089,62688,23193,DET,2018-04,01,52,Exports


In [8]:
# dictionary with preferred replacements for default column names obtained through API
column_names = {'CTY_CODE': 'COUNTRY_CODE',
                'ALL_VAL_MO': 'VALUE',
                'AIR_VAL_MO': 'AIR_VALUE',
                'AIR_WGT_MO': 'AIR_WEIGHT',
                'VES_VAL_MO': 'VESSEL_VALUE',
                'VES_WGT_MO': 'VESSEL_WEIGHT',
                'E_COMMODITY': 'COMMODITY_CODE',
                'I_COMMODITY': 'COMMODITY_CODE',
                'DISTRICT': 'DISTRICT_CODE',
                'Imports': 'TYPE',
                'Exports': 'TYPE'}

# replace the default column names (which are now in the first row) with the preferred ones
data.iloc[0,:].replace(to_replace=column_names, inplace=True)
# take the first row, convert to upper case, create a list and assign as a header
data.columns = data.iloc[0].str.upper().tolist()
# delete the first row
data.drop(labels=0, inplace=True)

# drop the SUMMARY_LVL column
data.drop(labels='SUMMARY_LVL', axis=1, inplace=True)
# to maintain consistency and ensure all the entries are monthly data, delete rows where CTY_CODE = '-'
data = data[data['COUNTRY_CODE'] != '-']

# separate the TIME column into columns YEAR and MONTH
data[['YEAR','MONTH']] = data['TIME'].str.split("-", expand=True)
# delete the TIME column
data.drop(labels='TIME', axis=1, inplace=True)

# add country, commodity and district names to the dataset
data = pd.merge(left=data, right=countries, on='COUNTRY_CODE', how='left')
data = pd.merge(left=data, right=commodities, on='COMMODITY_CODE', how='left')
data = pd.merge(left=data, right=districts, on='DISTRICT_CODE', how='left')

# convert numerical parameters to int
num_params = ['VALUE', 'AIR_VALUE', 'AIR_WEIGHT', 'VESSEL_VALUE', 'VESSEL_WEIGHT']
for param in num_params:
    data[param] = pd.to_numeric(data[param], errors='coerce')

One way to use the data is to export it in `csv` format.

In [9]:
data.to_csv('C:/Users/Berina/Desktop/Trade Data - API/monthly_trade_data.csv')

***

Depending on the end use, data can be aggregated in many different ways. One way is to get yearly data.

In [14]:
data.columns

Index(['COUNTRY_CODE', 'VALUE', 'AIR_VALUE', 'AIR_WEIGHT', 'VESSEL_VALUE',
       'VESSEL_WEIGHT', 'COMMODITY_CODE', 'DISTRICT_CODE', 'EXPORTS', 'YEAR',
       'MONTH', 'COUNTRY', 'COMMODITY', 'DISTRICT'],
      dtype='object')

In [19]:
yearly = data.groupby(['COUNTRY', 'COMMODITY', 'DISTRICT', 'EXPORTS', 'YEAR'], as_index=False) \
                      [['VALUE', 'AIR_VALUE', 'AIR_WEIGHT', 'VESSEL_VALUE', 'VESSEL_WEIGHT']].sum()

In [20]:
yearly.head()

,COUNTRY,COMMODITY,DISTRICT,EXPORTS,YEAR,VALUE,AIR_VALUE,AIR_WEIGHT,VESSEL_VALUE,VESSEL_WEIGHT
0,Afghanistan,Live Animals,"WASHINGTON, DC",Exports,2018,54000,54000,840,0,0
1,Afghanistan,Live Animals,"WASHINGTON, DC",Exports,2019,12000,12000,245,0,0
2,Afghanistan,Meat and Edible Meat Products,"HOUSTON-GALVESTON, TX",Exports,2018,836482,0,0,836482,1179302
3,Afghanistan,Meat and Edible Meat Products,"HOUSTON-GALVESTON, TX",Exports,2019,216059,0,0,216059,249392
4,Albania,Meat and Edible Meat Products,"HOUSTON-GALVESTON, TX",Exports,2018,827177,0,0,827177,948330


Let's also add another 

In [21]:
yearly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2587 entries, 0 to 2586
Data columns (total 10 columns):
COUNTRY          2587 non-null object
COMMODITY        2587 non-null object
DISTRICT         2587 non-null object
EXPORTS          2587 non-null object
YEAR             2587 non-null object
VALUE            2587 non-null int64
AIR_VALUE        2587 non-null int64
AIR_WEIGHT       2587 non-null int64
VESSEL_VALUE     2587 non-null int64
VESSEL_WEIGHT    2587 non-null int64
dtypes: int64(5), object(5)
memory usage: 222.3+ KB


In [23]:
countries

,COUNTRY_CODE,COUNTRY
0,5310,Afghanistan
1,4810,Albania
2,7210,Algeria
3,9510,American Samoa
4,4271,Andorra
5,7620,Angola
6,2481,Anguilla
7,2484,Antigua and Barbuda
8,3570,Argentina
9,4631,Armenia


In [24]:
yearly[yearly['COUNTRY'] == 'Chad']

,COUNTRY,COMMODITY,DISTRICT,EXPORTS,YEAR,VALUE,AIR_VALUE,AIR_WEIGHT,VESSEL_VALUE,VESSEL_WEIGHT
